In [ ]:
import sys
import torch
import torchvision

from PIL import Image
from torchvision import transforms

# Focus: understand transfer learning with ResNet—how to load a pretrained model, preprocess input,
# move data/model to the correct device, and run inference.

# Load pretrained ResNet-50 and set to eval
model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
)
model.eval()

# Image I/O & preprocessing
filename = "cat.jpg"  # change to your image if needed
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # shape: [1, 3, 224, 224]

# Device selection & placement
# TODO: Detect available device ("cuda" > "mps" > "cpu") and set `device`

# TODO: Move `input_batch` to `device`

# TODO: Move `model` to `device`

# Inference
with torch.no_grad():
    output = model(input_batch)  # logits shape: [1, 1000]

# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Read ImageNet 1K class names
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Show top-5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())